# Data Exploration

We recorded some data and want to check out if it's usable for data analysis.

Let's start with importing some plotting functions (don't care about the warning ... we should use something else, but this is just easier, for the time being).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

#use a nicer plotting style
plt.style.use(u'seaborn-talk')
print(plt.style.available)

## Walking versus not Walking
reading in accelerometer data from the file

In [ ]:
data = pd.read_csv('./data/2017-08-08 02.37.46 +0000_Accel.txt',delimiter='\t',skiprows=17,names=['time','skipped','ax','ay','az','label'])


In [ ]:
data[['ax','label']].plot()


In [4]:
accel_walking = data[data['label']==1][['ax','ay','az']]
accel_other = data[data['label']==0][['ax','ay','az']]


In [ ]:
#calculating a sliding window ...
mean_walking = accel_walking.rolling(200).apply(var).dropna()
mean_other = accel_other.rolling(200).apply(var).dropna()



In [ ]:
plot(mean_walking['ax'], mean_walking['ay'], '*')
plot(mean_other['ax'], mean_other['ay'],'*r')


## Motion Data

The assembly.dat file contains a recording done from a assembly session.

A person was doing the following activities: 

* hammering in nails (label 1)
* screwdring (label 2)
* sandpapering (label 3)
* sawing (label 4)

Label 0 is for doing none of the activities above. The sensor is attached to the subjects right wrist (x axis pointing towards the fingers). 
The sensor was sampled with 100 Hz, it's raw sensor data (not calibrated). The data structure of the file is shown in the header of the file.

## Import the data

The file uses tabs as separators and we need to skip 11 rows (the description of the content). 
Also we need to give the desciription for each column in the names variable.

In [7]:
data = pd.read_csv('./data/assembly.dat',delimiter='\t',skiprows=11,names=['s','usec','ax','ay','az','gx','gy','gz','mx','my','mz','label'])

In [ ]:
#to get an overview of the data you can use describe in pandas
data[['ax','ay','az']].describe()

In [ ]:
data[['gx','gy','gz']].plot()

In [ ]:
data[['ax','ay','az']].plot()

# Feature Caluclation and Selection
In the next step we will take the gyro data and calculate some features on the them.

In [ ]:
gyro_hammer = data[data['label']==1][['gx','gy','gz']]
gyro_screw = data[data['label']==2][['gx','gy','gz']]
gyro_sand = data[data['label']==3][['gx','gy','gz']]
gyro_saw = data[data['label']==4][['gx','gy','gz']]

In [ ]:
gyro_hammer.plot()
gyro_screw.plot()
gyro_sand.plot()
gyro_saw.plot()

In [ ]:
from numpy import mean

method = mean
wsize = 25

feat1 = gyro_screw.rolling(wsize).apply(mean).dropna()
feat2 = gyro_hammer.rolling(wsize).apply(mean).dropna()
feat3 = gyro_sand.rolling(wsize).apply(mean).dropna()
feat4 = gyro_saw.rolling(wsize).apply(mean).dropna()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import scatter
scatter(feat1['gz'], feat1['gy'])
scatter(feat2['gz'], feat2['gy'],color='red')
scatter(feat3['gz'], feat3['gy'],color='green')
scatter(feat4['gz'], feat4['gy'],color='yellow')

# Classification
In the following we will try how good our features are to automatically classify the different activities.

In [15]:
#don't look here ... I split the data in test and training set
#bad code :)

l_1 = len(feat1)
l_2 = len(feat2)
l_3 = len(feat3)
l_4 = len(feat4)

#X = feat1.append(feat2).append(feat3).append(feat4)
#Y = [1.0] * l_1 + [2.0]* l_2 + [3] * l_3 + [4] * l_4

X = feat1[0:int(l_1/2)].append(feat2[0:int(l_2/2)]).append(feat3[0:int(l_3/2)]).append(feat4[0:int(l_4/2)])
Y = [1]*int(l_1/2) + [2]*int(l_2/2)+[3]* int(l_3/2) + [4]* int(l_4/2)

T = feat1[int(l_1/2):l_1].append(feat2[int(l_2/2):l_2]).append(feat3[int(l_3/2):l_3]).append(feat4[int(l_4/2):l_4])
t_gg = [1]*len(feat1[int(l_1/2):l_1]) + [2]*len(feat2[int(l_2/2):l_2]) + [3]*len(feat3[int(l_3/2):l_3]) + [4]*len(feat1[int(l_4/2):l_4])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
dt = DecisionTreeClassifier()

dt.fit(X,Y)
res = dt.predict(T)

import pylab

pylab.plot(res+0.2, 'r.')
pylab.plot(t_gg, 'b.')


print(dt.score(T, t_gg))


